In [ ]:
import numpy as np

from example_models import get_upper_glycolysis
from mxlpy import Simulator, plot
from mxlpy.fuzzy import ThompsonState, thompson_sampling

# Create artificial data
p_orig = {
    "k1": 0.25,
    "k2": 1.2,
    "k3": 1.6,
    "k3m": 0.5,
    "k4": 100,
    "k5": 1.3,
    "k6": 1.5,
    "k7": 2.5,
}
m = get_upper_glycolysis().update_parameters(p_orig)
data, _ = (
    Simulator(m)
    .simulate_time_course(np.linspace(0, 10, 11, dtype=float))
    .get_result()
    .unwrap_or_err()
)

# Perform sampling
state = thompson_sampling(
    m,
    data=data,
    state=ThompsonState.from_parameter_values(
        {
            "k1": np.geomspace(1e-1, 1e2, 21),
            "k2": np.geomspace(1e-1, 1e2, 21),
            "k3": np.geomspace(1e-1, 1e2, 21),
            "k3m": np.geomspace(1e-1, 1e2, 21),
            "k4": np.geomspace(1e-1, 1e2, 21),
            "k5": np.geomspace(1e-1, 1e2, 21),
            "k6": np.geomspace(1e-1, 1e2, 21),
            "k7": np.geomspace(1e-1, 1e2, 21),
        }
    ),
    rtol=0.11,
    n=1000,
    parallel=True,
)

In [ ]:
from scipy.interpolate import CubicSpline

# Visualize results
fig, axs = plot.grid_layout(len(state.state), n_cols=4)
ai = iter(axs)

for ax, (name, var) in zip(axs, state.state.items(), strict=False):
    ax.set(title=f"{name}")
    xticks = np.arange(len(var["x"]))

    ax.bar(xticks, var["success"] / var["fail"])
    ax.set_xticks(xticks)
    ax.set_xticklabels((f"{i:.0e}" for i in var["x"]), rotation=90)

    x_trans = CubicSpline(var["x"], xticks)
    x_data = x_trans(p_orig[name])
    y_data = ax.transData.inverted().transform(ax.transAxes.transform([0, 0.5]))[1]
    ax.annotate(
        p_orig[name],
        xy=(x_data, 0),  # type: ignore
        xytext=(x_data, y_data),  # type: ignore
        ha="center",
        arrowprops={"arrowstyle": "->"},
    )